In [1]:
import pandas as pd


In [5]:
transactions_df = pd.read_csv("transactions_tm1_e.csv")
customers_df = pd.read_csv("customers_tm1_e.csv")

In [6]:
transactions_df.head()

,date,account_id,customer_id,amount,transaction_date,deposit,withdrawal
0,2007-01-31,24137947,91,3034.26,2007-01-31,3034.26,0.00
1,2007-01-31,24137947,91,-5295.18,2007-01-16,0.00,-5295.18
2,2007-02-28,24137947,91,0.00,2007-02-28,0.00,0.00
3,2007-03-31,24137947,91,-0.00,2007-03-30,0.00,-0.00
4,2007-03-31,24137947,91,-0.00,2007-03-11,0.00,-0.00


In [49]:
account_ids = transactions_df.account_id.value_counts()

In [54]:
#number of account ids that have made transactions 
print(len(account_ids))

account_ids

116016


24143481    393
24148664    390
24138463    388
24147524    387
24148364    385
           ... 
24198347      1
24198375      1
24251181      1
24198388      1
24253962      1
Name: account_id, Length: 116016, dtype: int64

In [61]:
months = []
for n in transactions_df["date"]:
    months.append(n[:7])
transactions_df["month/year"] = months

In [62]:
transactions_df.head()

,date,account_id,customer_id,amount,transaction_date,deposit,withdrawal,month/year
0,2007-01-31,24137947,91,3034.26,2007-01-31,3034.26,0.00,2007-01
1,2007-01-31,24137947,91,-5295.18,2007-01-16,0.00,-5295.18,2007-01
2,2007-02-28,24137947,91,0.00,2007-02-28,0.00,0.00,2007-02
3,2007-03-31,24137947,91,-0.00,2007-03-30,0.00,-0.00,2007-03
4,2007-03-31,24137947,91,-0.00,2007-03-11,0.00,-0.00,2007-03


In [79]:
grp_df = transactions_df.groupby(['account_id', 'month/year', 'customer_id']).agg({'amount': ['sum']})
grp_df.columns = ['amount_sum']
grp_df = grp_df.reset_index()

## aggregated by month 

In [116]:
grp_df.head()

,account_id,month/year,customer_id,amount_sum
0,24137947,2007-01,91,-2260.92
1,24137947,2007-02,91,0.00
2,24137947,2007-03,91,0.00
3,24137948,2007-01,92,0.00
4,24137948,2007-02,92,1164.90


In [81]:
print(grp_df['month/year'].min())
print(grp_df['month/year'].max())
print(len(grp_df))

2007-01
2020-05
2129121


In [100]:
interest_rates = pd.read_csv("FEDFUNDS.csv")
interest_rates.head()

,DATE,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83


In [101]:
type(interest_rates['DATE'][0])

str

In [102]:
months = []
for n in interest_rates["DATE"]:
    months.append(n[:7])
interest_rates["month/year"] = months

In [108]:
interest_rates = interest_rates[["FEDFUNDS", "month/year"]]

In [109]:
interest_rates.head()

,FEDFUNDS,month/year
0,0.80,1954-07
1,1.22,1954-08
2,1.07,1954-09
3,0.85,1954-10
4,0.83,1954-11


In [119]:
grp_df = grp_df.join(interest_rates.set_index('month/year'), on='month/year', how='left', lsuffix='_tran', rsuffix='_fed')

## Added month's interest rate into table

In [120]:
grp_df.head()

,account_id,month/year,customer_id,amount_sum,FEDFUNDS
0,24137947,2007-01,91,-2260.92,5.25
1,24137947,2007-02,91,0.00,5.26
2,24137947,2007-03,91,0.00,5.26
3,24137948,2007-01,92,0.00,5.25
4,24137948,2007-02,92,1164.90,5.26
